In [175]:
from playwright.async_api import async_playwright

args = [
    "--disable-blink-features=AutomationControlled"
]
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False, args=args)
page = await browser.new_page()

In [176]:
from urllib.parse import parse_qs, urlparse
import hashlib
from pathlib import Path
import json

# If we've already added the listener, remove it
try:
    page.remove_listener('response', process_response)
except:
    pass

async def process_response(response):
    if response.ok and response.url.startswith("https://www.tiktok.com/api/"):
        try:
            parsed = urlparse(response.url)

            # Hash the query parameters and the response.text to make a unique filename
            m = hashlib.md5()
            m.update(parsed.query.encode('utf-8'))
            m.update((await response.text()).encode('utf-8'))
            filename = m.hexdigest()

            # the full path is based on the URL
            path = Path(parsed.path.lstrip("/")).joinpath(filename).with_suffix(".json")
            path.parent.mkdir(parents=True, exist_ok=True)

            print("Writing content to ", path)

            query = parse_qs(parsed.query)
            query = {k: v[0] for k, v in query.items()}

            content = {
                'url': response.url,
                'query': query,
                'headers': await response.all_headers(),
                'data': await response.json(),
            }

            path.write_text(json.dumps(content, indent=2))
        except Exception as e:
            print("Error processing response", e)

page.on("response", process_response)

Writing content to  api/search/suggest/guide/1921980b8c1a06f5213d7105883d68b5.json
Writing content to  api/policy/notice/76ba601e76d13a8ed36faf4824721208.json
Writing content to  api/ba/business/suite/permission/list/3469b6100caeec5994e7967e4905ea5f.json
Writing content to  api/compliance/settings/fa069ca879167b37e0958b17023a3938.json
Writing content to  api/search/user/preview/7a4d33589ad187eb896474afcddeda27.json
Writing content to  api/preload/item_list/e688e1f13696f589a16313a23845d169.json
Writing content to  api/ba/business/suite/permission/list/a0ae41e039832ce9dbb3a8c090d294fc.json
Writing content to  api/feedback/v1/newest_reply/05717fe4eb390a312673f57c95ea7b13.json
Writing content to  api/user/list/c742f2b7342e0b8be73373e98a7cb3b8.json
Writing content to  api/inbox/notice_count/568b318b645006024482a5d2eaf3d9ed.json
Writing content to  api/policy/notice/8cd62558c15bc430677272ec5cdd6b69.json
Writing content to  api/compliance/settings/7ffa34fad766a564089591ef132fb1e5.json
Writing

In [177]:
await browser.close()

In [2]:
import glob

filenames = glob.glob("api/post/item_list/*.json")
filenames[:5]

['api/post/item_list/5cb0c15c4fbe664ad4c272d68b8590e3.json',
 'api/post/item_list/6b45c2be1e07e9a5680bc5341fe54725.json',
 'api/post/item_list/3a215d61d9bf7dc60d7fe9622380a31b.json',
 'api/post/item_list/7ef63e219a428d8192393c3d2bf07266.json',
 'api/post/item_list/348b654d28f07521d668432862694ead.json']

In [29]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['itemList'])

len(rows)

5820

In [28]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,AIGCDescription,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,officalItem,originalItem,privateItem,secret,shareEnabled,stitchDisplay,stitchEnabled,textExtra,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_uniqueId,author_verified,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,music_playUrl,diversificationId,video_subtitleInfos,playlistId,anchors,adLabelVersion,adAuthorization,isAd,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,itemMute,poi_address,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_typeCode,BAInfo,isECVideo,music_scheduleSearchTime,duetInfo_duetFromId,videoSuggestWordsList_video_suggest_words_struct,warnInfo,isPinnedItem,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'How to make a handmade bow! #wrappinggifts #wrappingpresents...,1637693876,How to make a handmade bow! #wrappinggifts #wrappingpresents #giftwrapp...,False,0,True,False,7033841630654549294,0,False,False,False,False,True,0,True,"[{'awemeId': '', 'end': 42, 'hashtagName': 'wrappinggifts', 'isCommerce...",https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,0,0,0,False,6875149358373176325,False,False,Margaret & Stetson,False,False,0,MS4wLjABAAAAdK-fCBiHfDnxlcqWI8k0t8JSvC2zZ2elAJFE2WLjwpFmLRrGIHCVo8_e6Ww...,False,"Mother-daughter duo\nAmazon finds, home organization, & more\nSHOP OUR ...",0,everythingenvy,False,True,We Wish You a Merry Christmas,Merry Christmas,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,29.0,6709421144174233602,False,We Wish You a Merry Christmas,121,4,630,17000,67,121,4,630,17000,0,67,0,1453478.0,"[{'Bitrate': 1453478, 'CodecType': 'h264', 'GearName': 'normal_540_0', ...",h264,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5a1b8d9d39e...,540p,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,12,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,,normal,mp4,1024,7033841630654549294,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,540p,normal,-12.9,0.89125,576,https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/

In [29]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

5356 total videos


,tiktok_url,AIGCDescription,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,officalItem,originalItem,privateItem,secret,shareEnabled,stitchDisplay,stitchEnabled,textExtra,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_uniqueId,author_verified,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,music_playUrl,diversificationId,video_subtitleInfos,playlistId,anchors,adLabelVersion,adAuthorization,isAd,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,itemMute,poi_address,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_typeCode,BAInfo,isECVideo,music_scheduleSearchTime,duetInfo_duetFromId,videoSuggestWordsList_video_suggest_words_struct,warnInfo,isPinnedItem,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,https://tiktok.com/@everythingenvy/video/7033841630654549294,,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'How to make a handmade bow! #wrappinggifts #wrappingpresents...,1637693876,How to make a handmade bow! #wrappinggifts #wrappingpresents #giftwrapp...,False,0,True,False,7033841630654549294,0,False,False,False,False,True,0,True,"[{'awemeId': '', 'end': 42, 'hashtagName': 'wrappinggifts', 'isCommerce...",https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,0,0,0,False,6875149358373176325,False,False,Margaret & Stetson,False,False,0,MS4wLjABAAAAdK-fCBiHfDnxlcqWI8k0t8JSvC2zZ2elAJFE2WLjwpFmLRrGIHCVo8_e6Ww...,False,"Mother-daughter duo\nAmazon finds, home organization, & more\nSHOP OUR ...",0,everythingenvy,False,True,We Wish You a Merry Christmas,Merry Christmas,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,29.0,6709421144174233602,False,We Wish You a Merry Christmas,121,4,630,17000,67,121,4,630,17000,0,67,0,1453478.0,"[{'Bitrate': 1453478, 'CodecType': 'h264', 'GearName': 'normal_540_0', ...",h264,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5a1b8d9d39e...,540p,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,12,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,,normal,mp4,1024,7033841630654549294,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,540p,normal,-1

In [30]:
videos_df['author_nickname'].value_counts()

author_nickname
AMAZON FINDS           1255
Sarah Lindner          1144
Jared Andersen         1092
Margaret & Stetson      796
Sam Shan Shops 🦋        487
Anya                    264
Justice Buys            155
TONY STATOVCI           136
reb for the rebrand      27
Name: count, dtype: int64

In [31]:
filtered_df = videos_df[videos_df['author_nickname'] == "reb for the rebrand"]

In [32]:
filtered_df.to_csv("rebrebrand_videos.csv", index=False)

In [34]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("rebrebrand_videos.csv")

df.head()


,tiktok_url,AIGCDescription,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,officalItem,originalItem,privateItem,secret,shareEnabled,stitchDisplay,stitchEnabled,textExtra,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_uniqueId,author_verified,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,music_playUrl,diversificationId,video_subtitleInfos,playlistId,anchors,adLabelVersion,adAuthorization,isAd,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,itemMute,poi_address,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_typeCode,BAInfo,isECVideo,music_scheduleSearchTime,duetInfo_duetFromId,videoSuggestWordsList_video_suggest_words_struct,warnInfo,isPinnedItem,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,https://tiktok.com/@rebmasel/video/7077545446637440299,NaN,"[{'coverLarger': '', 'coverMedium': '', 'coverThumb': '', 'desc': '', '...",False,"[{'desc': 'oven timer was a paid actor #funeral', 'textExtra': [{'aweme...",1647869464,oven timer was a paid actor #funeral,False,0,True,False,7077545446637440299,0,False,False,False,False,True,0,True,"[{'awemeId': '', 'end': 36, 'hashtagName': 'funeral', 'isCommerce': Fal...",https://p19-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/7313960034524...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/7313960034524...,https://p19-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/7313960034524...,0,0,0,False,6726756092489237509,False,False,reb for the rebrand,False,False,0,MS4wLjABAAAAhhKOPVZobXJu1z37ZiUlh2_rfiOnvqxOb7FWeThVSi5e-Exqwf9EsOJW7Qk...,False,29\nJ.D./Attorney/host of @rebuttalpod ❤️‍🔥\nrebmasel@viralnationtalent...,0,rebmasel,False,True,The Journey to Find Yourself,Bobby Cole,https://p16-sg.tiktokcdn.com/aweme/720x720/tos-alisg-v-2774/owADiqdHIcc...,https://p16-sg.tiktokcdn.com/aweme/200x200/tos-alisg-v-2774/owADiqdHIcc...,https://p16-sg.tiktokcdn.com/aweme/100x100/tos-alisg-v-2774/owADiqdHIcc...,60.0,6716039644095465474,False,Funeral Music,71700,4148,2200000,10000000,21000,71653,4148,2200000,10000000,0,21000,55.66,1026810.0,"[{'Bitrate': 1026810, 'CodecType': 'h264', 'GearName': 'normal_540_0', ...",h264,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,540p,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-pv...,56,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,NaN,normal,mp4,1024,7077545446637440299,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,540p,normal,-16.4,0.83176,576,https://p16-p

In [35]:
df.shape

(27, 129)

In [36]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows())[:20], desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', row['author_uniqueId'], "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


# downloading hashtags #amazonfinds

In [66]:
# Just grab the first one
filename = glob.glob("api/search/item/full/*.json")[-1]

IndexError: list index out of range

In [47]:
from pprint import pprint

with open(filename) as fp:
    content = json.load(fp)

content.keys()

dict_keys(['url', 'query', 'headers', 'data'])

In [50]:
pprint(content['data']['item_list'][0])

{'anchors': [{'description': 'CapCut · Video Editor',
              'extraInfo': {'subtype': ''},
              'icon': {'urlList': ['https://p16-sg.tiktokcdn.com/obj/tiktok-obj/capcut_logo_64px_bk.png?nonce=24091&refresh_token=a89d4cce4731c28497ca5e20c59183b6&shp=b59d6b55&shcp=-']},
              'id': '0',
              'keyword': 'CapCut · Edit like a pro',
              'logExtra': '{"anchor_id":0,"anchor_name":"CapCut · Edit like a '
                          'pro","anchor_title_detail":"None","anchor_type":"TT_CAPCUT","capability_key":"general","is_two_line":0,"maker_source":"","template_id":"none","tutorial_id":"none","video_source":1,"video_type_id":1}',
              'schema': 'https://www.capcut.com/editor?scenario=tiktok&utm_campaign=71351105&utm_medium=tiktok_anchor&utm_source=tiktok_anchor_tool',
              'thumbnail': {'height': 64,
                            'urlList': ['https://p16-sg.tiktokcdn.com/obj/tiktok-obj/64x_Capcut3x.png?nonce=45473&refresh_token=d11bb1f21

In [51]:
# Just grab the first one
filenames = glob.glob("api/search/item/full/*.json")
filenames[:5]

['api/search/item/full/6b7bf35ee5e89e60fc3ec605b17f29b6.json',
 'api/search/item/full/693120cf8507d9268155c1d04f40cebb.json',
 'api/search/item/full/b09165a742af1ae042ab9f5a6c63cd75.json',
 'api/search/item/full/8297c39452c1326d92a43a07cda45648.json',
 'api/search/item/full/cc943d7da34ccbdffb6d2f96245c69c0.json']

In [52]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['item_list'])

len(rows)

108

In [53]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,effectStickers,stickersOnItem,anchors
0,7264730735444053290,Number 4 is my fav 😅🫶🏼 #amazonfinds #justicebuys #amazonmusthaves #prod...,1691451941,"[{'id': '1604894787222533', 'title': 'amazonfinds', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 26, 'end': 38, 'hashtagName': 'amazonfinds', ...",False,False,False,0,False,False,False,False,True,True,True,False,False,7264730735444053290,1024,576,66,540p,https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/e11e697b0e63449...,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/8fff5e2705d...,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/7ffff24ae04...,https://v16-webapp-prime.tiktok.com/video/tos/maliva/tos-maliva-ve-0068...,https://v16-webapp-prime.tiktok.com/video/tos/maliva/tos-maliva-v-0068c...,"[, https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/8fff5e2705d8...",https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/e11e697b0e63449...,1466243,normal,mp4,normal,,7155954783169496106,justicebuys,Justice Buys,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,Product reviews 📦\nAmazon finds 🔥\nAll products in videos ⬇️,False,MS4wLjABAAAAPJ61j_yk0kkKnyG0Jqe0echqNgDDRYnLqxLh05cjWfMtD2my-fYMetcEaZz...,False,False,0,False,0,0,0,False,0,7264730743132228394,original sound,,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,Justice Buys,True,66,,1800000,79900,8095,22000000,353600,0,65,1200000,35900000,163,1358,35900000,NaN,NaN,NaN
1,7225198231125380378,hot girl amazon self care must haves🧖🏼‍♀️ follow for more💌 #amazonfinds...,1682247564,"[{'id': '1604894787222533', 'title': 'amazonfinds', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 62, 'end': 74, 'hashtagName': 'amazonfinds', ...",False,False,False,0,False,False,False,False,True,True,True,False,False,7225198231125380378,1024,576,15,540p,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/oI...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/58...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/b0...,https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pve...,https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pv-...,"[, https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/580...",https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/oINI4R...,283186,normal,mp4,normal,,7220878699334222853,styleinspo.4you,styleinspo.4you,https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-

In [54]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

108 total videos


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,effectStickers,stickersOnItem,anchors
0,https://tiktok.com/@justicebuys/video/7264730735444053290,7264730735444053290,Number 4 is my fav 😅🫶🏼 #amazonfinds #justicebuys #amazonmusthaves #prod...,1691451941,"[{'id': '1604894787222533', 'title': 'amazonfinds', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 26, 'end': 38, 'hashtagName': 'amazonfinds', ...",False,False,False,0,False,False,False,False,True,True,True,False,False,7264730735444053290,1024,576,66,540p,https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/e11e697b0e63449...,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/8fff5e2705d...,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/7ffff24ae04...,https://v16-webapp-prime.tiktok.com/video/tos/maliva/tos-maliva-ve-0068...,https://v16-webapp-prime.tiktok.com/video/tos/maliva/tos-maliva-v-0068c...,"[, https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/8fff5e2705d8...",https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/e11e697b0e63449...,1466243,normal,mp4,normal,,7155954783169496106,justicebuys,Justice Buys,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,Product reviews 📦\nAmazon finds 🔥\nAll products in videos ⬇️,False,MS4wLjABAAAAPJ61j_yk0kkKnyG0Jqe0echqNgDDRYnLqxLh05cjWfMtD2my-fYMetcEaZz...,False,False,0,False,0,0,0,False,0,7264730743132228394,original sound,,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,Justice Buys,True,66,,1800000,79900,8095,22000000,353600,0,65,1200000,35900000,163,1358,35900000,NaN,NaN,NaN
1,https://tiktok.com/@styleinspo.4you/video/7225198231125380378,7225198231125380378,hot girl amazon self care must haves🧖🏼‍♀️ follow for more💌 #amazonfinds...,1682247564,"[{'id': '1604894787222533', 'title': 'amazonfinds', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 62, 'end': 74, 'hashtagName': 'amazonfinds', ...",False,False,False,0,False,False,False,False,True,True,True,False,False,7225198231125380378,1024,576,15,540p,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/oI...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/58...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/b0...,https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pve...,https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pv-...,"[, https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/580...",https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/oINI4R...,

In [55]:
# export videos_df as csv
videos_df.to_csv("search_amazonfinds_videos.csv", index=False)

In [56]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("search_amazonfinds_videos.csv")

df.head()


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,effectStickers,stickersOnItem,anchors
0,https://tiktok.com/@justicebuys/video/7264730735444053290,7264730735444053290,Number 4 is my fav 😅🫶🏼 #amazonfinds #justicebuys #amazonmusthaves #prod...,1691451941,"[{'id': '1604894787222533', 'title': 'amazonfinds', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 26, 'end': 38, 'hashtagName': 'amazonfinds', ...",False,False,False,0,False,False,False,False,True,True,True,False,False,7264730735444053290,1024,576,66,540p,https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/e11e697b0e63449...,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/8fff5e2705d...,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/7ffff24ae04...,https://v16-webapp-prime.tiktok.com/video/tos/maliva/tos-maliva-ve-0068...,https://v16-webapp-prime.tiktok.com/video/tos/maliva/tos-maliva-v-0068c...,"['', 'https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/8fff5e270...",https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/e11e697b0e63449...,1466243,normal,mp4,normal,NaN,7155954783169496106,justicebuys,Justice Buys,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,Product reviews 📦\nAmazon finds 🔥\nAll products in videos ⬇️,False,MS4wLjABAAAAPJ61j_yk0kkKnyG0Jqe0echqNgDDRYnLqxLh05cjWfMtD2my-fYMetcEaZz...,False,False,0,False,0,0,0,False,0,7264730743132228394,original sound,NaN,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/73159300483666411...,Justice Buys,True,66,NaN,1800000,79900,8095,22000000,353600,0,65,1200000,35900000,163,1358,35900000,NaN,NaN,NaN
1,https://tiktok.com/@styleinspo.4you/video/7225198231125380378,7225198231125380378,hot girl amazon self care must haves🧖🏼‍♀️ follow for more💌 #amazonfinds...,1682247564,"[{'id': '1604894787222533', 'title': 'amazonfinds', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 62, 'end': 74, 'hashtagName': 'amazonfinds', ...",False,False,False,0,False,False,False,False,True,True,True,False,False,7225198231125380378,1024,576,15,540p,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/oI...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/58...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/b0...,https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pve...,https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pv-...,"['', 'https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/...",https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/o

In [57]:
df.shape

(108, 81)

In [60]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows())[:50], desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', "amazonfinds", "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


ERROR: [TikTok] 7138883978441788718: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7138883978441788718: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@alixearle/video/7138883978441788718


In [61]:
all_videos = glob.glob("videos/amazonfinds/*.mp4", recursive=True)
len(all_videos)

49

# downloading hashtags #grwm

In [6]:
import glob

In [78]:
# Just grab the first one
filename = glob.glob("api/search/item/full/*.json")[-1]
filename

'api/search/item/full/f0a33449569c921104aab73e5ced9ce5.json'

In [79]:
from pprint import pprint

with open(filename) as fp:
    content = json.load(fp)

content.keys()

dict_keys(['url', 'query', 'headers', 'data'])

In [80]:
pprint(content['data']['item_list'][0])

{'author': {'avatarLarger': 'https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/f6c3d3b5f69a20e03665dede08e39901~c5_1080x1080.jpeg?lk3s=a5d48078&nonce=40285&refresh_token=c633a32331afa322abe25d1fd491d860&x-expires=1721779200&x-signature=6ckO16hOo%2FjDFmQTYK628IMmVAQ%3D&shp=a5d48078&shcp=b59d6b55',
            'avatarMedium': 'https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/f6c3d3b5f69a20e03665dede08e39901~c5_720x720.jpeg?lk3s=a5d48078&nonce=84718&refresh_token=bd92930dbd45c9a10a9a8037bef2e53a&x-expires=1721779200&x-signature=VNJ419YiIRaH9%2FvNrTdrx%2BeLSHE%3D&shp=a5d48078&shcp=b59d6b55',
            'avatarThumb': 'https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/f6c3d3b5f69a20e03665dede08e39901~c5_100x100.jpeg?lk3s=a5d48078&nonce=18585&refresh_token=ec90a988ff26d27ccf56dcf67b8da8c9&x-expires=1721779200&x-signature=WQaQHsxQfY3PVNIaOxfeVvtBEkc%3D&shp=a5d48078&shcp=b59d6b55',
            'commentSetting': 0,
            'download

In [84]:
# Just grab the first one
filenames = glob.glob("api/search/item/full/*.json")
filenames[:5]

['api/search/item/full/61cb918d67bec63333f1e3ec0eeef31e.json', 'api/search/item/full/30f4fdb455077752f71bd3d7ded6f143.json', 'api/search/item/full/2f9fb21af6921ddb3d975319cc5c6b0f.json', 'api/search/item/full/a01db4701a7f03cb6fec44aab5df89bc.json', 'api/search/item/full/492d73728e86f1eadc0757f920ff0df7.json']

In [86]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['item_list'])

len(rows)

152

In [88]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,stickersOnItem,effectStickers,anchors,videoSuggestWordsList_video_suggest_words_struct
0,7305165993272200490,Replying to @🇧🇷Gabriela🇮🇹 I actually love the ice facial #grwm #Grwm #m...,1700866524,"[{'id': '72878', 'title': 'grwm', 'desc': 'Share your #GRWM routine!', ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 29, 'hashtagName': '', 'hashtagId'...",False,False,False,0,False,False,False,False,True,True,True,False,False,7305165993272200490,1024,576,59,540p,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5b90c465f72...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"[, https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/5b90c465f723...",https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/1b2e...,888742,normal,mp4,normal,,7253200608634635306,hallie.ganga,🔆hallie!🌴,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,Parent managed account!\nmy back up!➡️ @Hallie🍋‍🟩 \n💌hallieganga11@gmai...,False,MS4wLjABAAAA4HzYuG13V-zlvTZSy3NKRmtZv3-iL8jgkArKW25iH-xT50dXH90UA_ZUkK6...,False,False,0,False,0,1,1,False,3,7305166086930991915,original sound,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-v-...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,🔆hallie!🌴,True,59,,169700,275,537,1800000,15700,0,83,426200,19700000,100,3034,19700000,NaN,NaN,NaN,NaN
1,7342573657002233130,🚽 Rules 🫠 #grwm #getreadywithme #school #schoolrules #grwmforschool #re...,1709576205,"[{'id': '72878', 'title': 'grwm', 'desc': 'Share your #GRWM routine!', ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 17, 'hashtagName': 'grwm', 'hashta...",False,False,False,0,False,False,False,False,True,True,True,False,False,7342573657002233130,1024,576,97,540p,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-pv...,"[, https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/2...",https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/owEY...,174

In [89]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

152 total videos


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,stickersOnItem,effectStickers,anchors,videoSuggestWordsList_video_suggest_words_struct
0,https://tiktok.com/@hallie.ganga/video/7305165993272200490,7305165993272200490,Replying to @🇧🇷Gabriela🇮🇹 I actually love the ice facial #grwm #Grwm #m...,1700866524,"[{'id': '72878', 'title': 'grwm', 'desc': 'Share your #GRWM routine!', ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 29, 'hashtagName': '', 'hashtagId'...",False,False,False,0,False,False,False,False,True,True,True,False,False,7305165993272200490,1024,576,59,540p,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5b90c465f72...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"[, https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/5b90c465f723...",https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/1b2e...,888742,normal,mp4,normal,,7253200608634635306,hallie.ganga,🔆hallie!🌴,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,Parent managed account!\nmy back up!➡️ @Hallie🍋‍🟩 \n💌hallieganga11@gmai...,False,MS4wLjABAAAA4HzYuG13V-zlvTZSy3NKRmtZv3-iL8jgkArKW25iH-xT50dXH90UA_ZUkK6...,False,False,0,False,0,1,1,False,3,7305166086930991915,original sound,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-v-...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,🔆hallie!🌴,True,59,,169700,275,537,1800000,15700,0,83,426200,19700000,100,3034,19700000,NaN,NaN,NaN,NaN
1,https://tiktok.com/@krissleos/video/7342573657002233130,7342573657002233130,🚽 Rules 🫠 #grwm #getreadywithme #school #schoolrules #grwmforschool #re...,1709576205,"[{'id': '72878', 'title': 'grwm', 'desc': 'Share your #GRWM routine!', ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 17, 'hashtagName': 'grwm', 'hashta...",False,False,False,0,False,False,False,False,True,True,True,False,False,7342573657002233130,1024,576,97,540p,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-pv...,"[, https://p16-pu-sign-useas

In [90]:
# export videos_df as csv
videos_df.to_csv("search_grwm_videos.csv", index=False)

In [91]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("search_grwm_videos.csv")

df.head()


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,stickersOnItem,effectStickers,anchors,videoSuggestWordsList_video_suggest_words_struct
0,https://tiktok.com/@hallie.ganga/video/7305165993272200490,7305165993272200490,Replying to @🇧🇷Gabriela🇮🇹 I actually love the ice facial #grwm #Grwm #m...,1700866524,"[{'id': '72878', 'title': 'grwm', 'desc': 'Share your #GRWM routine!', ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 29, 'hashtagName': '', 'hashtagId'...",False,False,False,0,False,False,False,False,True,True,True,False,False,7305165993272200490,1024,576,59,540p,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5b90c465f72...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"['', 'https://p16-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/5b90c465f...",https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/1b2e...,888742,normal,mp4,normal,NaN,7253200608634635306,hallie.ganga,🔆hallie!🌴,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,Parent managed account!\nmy back up!➡️ @Hallie🍋‍🟩 \n💌hallieganga11@gmai...,False,MS4wLjABAAAA4HzYuG13V-zlvTZSy3NKRmtZv3-iL8jgkArKW25iH-xT50dXH90UA_ZUkK6...,False,False,0,False,0,1,1,False,3,7305166086930991915,original sound,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-v-...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/02...,🔆hallie!🌴,True,59,NaN,169700,275,537,1800000,15700,0,83,426200,19700000,100,3034,19700000,NaN,NaN,NaN,NaN
1,https://tiktok.com/@krissleos/video/7342573657002233130,7342573657002233130,🚽 Rules 🫠 #grwm #getreadywithme #school #schoolrules #grwmforschool #re...,1709576205,"[{'id': '72878', 'title': 'grwm', 'desc': 'Share your #GRWM routine!', ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 17, 'hashtagName': 'grwm', 'hashta...",False,False,False,0,False,False,False,False,True,True,True,False,False,7342573657002233130,1024,576,97,540p,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-pv...,"['', 'https://p16-pu-s

In [92]:
df.shape

(152, 82)

In [101]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows())[100:], desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', "grwm", "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


In [103]:
all_videos = glob.glob("videos/grwm/*.mp4", recursive=True)
len(all_videos)

82

# downloading #bookrecs

In [107]:
# Just grab the first one
filename = glob.glob("api/challenge/item_list/*.json")[-1]
filename

'api/challenge/item_list/d1f5177b1b62f794f3b4e75671ac079e.json'

In [108]:
from pprint import pprint

with open(filename) as fp:
    content = json.load(fp)

content.keys()

dict_keys(['url', 'query', 'headers', 'data'])

In [126]:
pprint(content['data']['itemList'][0])

[{'AIGCDescription': '',
  'BAInfo': '',
  'adAuthorization': False,
  'adLabelVersion': 0,
  'aigcLabelType': 0,
  'author': {'avatarLarger': 'https://p16-sign-sg.tiktokcdn.com/aweme/1080x1080/tos-alisg-avt-0068/7331430697380347924.jpeg?lk3s=a5d48078&nonce=91556&refresh_token=d1112cefd6538a73ffbfb8522fea07c0&x-expires=1721858400&x-signature=0PAbThL2UokUQdOYTgKewfOoPaM%3D&shp=a5d48078&shcp=b59d6b55',
             'avatarMedium': 'https://p16-sign-sg.tiktokcdn.com/aweme/720x720/tos-alisg-avt-0068/7331430697380347924.jpeg?lk3s=a5d48078&nonce=50988&refresh_token=8e45c52e32129dfc63b8d9d236c40e79&x-expires=1721858400&x-signature=PTXZ%2B4JjAkFhVDcDYZKVbdiOUL4%3D&shp=a5d48078&shcp=b59d6b55',
             'avatarThumb': 'https://p16-sign-sg.tiktokcdn.com/aweme/100x100/tos-alisg-avt-0068/7331430697380347924.jpeg?lk3s=a5d48078&nonce=69248&refresh_token=5b8310ad89cf78b26405aabbfbc5a9fc&x-expires=1721858400&x-signature=clmdrTmVKiiys7%2B0bSaTs2aKrVE%3D&shp=a5d48078&shcp=b59d6b55',
             'com

In [115]:
# Just grab the first one
filenames = glob.glob("api/challenge/item_list/*.json")
filenames[:5]

['api/challenge/item_list/5a4f53df7dc196ba926c6e3555777a0f.json', 'api/challenge/item_list/1731d9ae0b63e40de4790a2825b04606.json', 'api/challenge/item_list/11cbef210aeaa5d7dac1ebf8809e9d40.json', 'api/challenge/item_list/3af7b73ee7817fbfb2d6a60f8a68984f.json', 'api/challenge/item_list/db1dadea1cf231274a92ce37ac66297e.json']

In [127]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['itemList'])

len(rows)

168

In [129]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,AIGCDescription,BAInfo,adAuthorization,adLabelVersion,aigcLabelType,challenges,collected,contents,createTime,desc,digged,diversificationId,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,itemMute,officalItem,originalItem,playlistId,privateItem,secret,shareEnabled,showNotPass,stickersOnItem,stitchDisplay,stitchEnabled,textExtra,vl1,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_ttSeller,author_uniqueId,author_verified,authorStats_diggCount,authorStats_followerCount,authorStats_followingCount,authorStats_friendCount,authorStats_heart,authorStats_heartCount,authorStats_videoCount,duetInfo_duetFromId,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_playUrl,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_reflowCover,video_shareCover,video_size,video_subtitleInfos,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,videoSuggestWordsList_video_suggest_words_struct,anchors,effectStickers,poi_address,poi_category,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiId,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_type,poi_typeCode,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,,,False,0,0,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'these books > 💖 #bookrecommendations #bookstan #booktok #boo...,1711556065,these books > 💖 #bookrecommendations #bookstan #booktok #bookstagram #b...,False,10091.0,0,True,False,7351077151376002347,False,0,False,False,False,7262051690747120430,False,False,True,False,"[{'stickerText': ['books i think you should read at least once ♥️'], 's...",0,True,"[{'awemeId': '', 'end': 37, 'hashtagId': '58993759', 'hashtagName': 'bo...",False,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,0,0,0,False,7085887049387099179,False,False,adridiaries 💌,False,False,0,MS4wLjABAAAAGyg5PhHBHSGFFuL2pbLBJiXdawbgLOtOxUidLwTLzZWOXOHXYDp5IrrFpnC...,False,digital diary\n🧸🎃🍂☕️🕯️🧦🎧\n💌 asosa4994@gmail.com,0,False,adridiaries,False,40100,308300,314,0,18900000,18900000,245,0,True,,scarlett✨,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,75,7047276707090270981,True,https://v16m.tiktokcdn-us.com/00d74933a643f72aa62b712fd83bbe04/669f3473...,hardins letter,466200,3049,1400000,8000000,19300,466162,3049,1385711,7979916,0,19332,63.36,1155035.0,"[{'Bitrate': 1155035, 'CodecType': 'h264', 'GearName': 'normal_540_0', ...",h264,https://p19-pu-sign-useast8.tikt

In [130]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

167 total videos


,tiktok_url,AIGCDescription,BAInfo,adAuthorization,adLabelVersion,aigcLabelType,challenges,collected,contents,createTime,desc,digged,diversificationId,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,itemMute,officalItem,originalItem,playlistId,privateItem,secret,shareEnabled,showNotPass,stickersOnItem,stitchDisplay,stitchEnabled,textExtra,vl1,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_ttSeller,author_uniqueId,author_verified,authorStats_diggCount,authorStats_followerCount,authorStats_followingCount,authorStats_friendCount,authorStats_heart,authorStats_heartCount,authorStats_videoCount,duetInfo_duetFromId,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_playUrl,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_reflowCover,video_shareCover,video_size,video_subtitleInfos,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,videoSuggestWordsList_video_suggest_words_struct,anchors,effectStickers,poi_address,poi_category,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiId,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_type,poi_typeCode,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,https://tiktok.com/@adridiaries/video/7351077151376002347,,,False,0,0,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'these books > 💖 #bookrecommendations #bookstan #booktok #boo...,1711556065,these books > 💖 #bookrecommendations #bookstan #booktok #bookstagram #b...,False,10091.0,0,True,False,7351077151376002347,False,0,False,False,False,7262051690747120430,False,False,True,False,"[{'stickerText': ['books i think you should read at least once ♥️'], 's...",0,True,"[{'awemeId': '', 'end': 37, 'hashtagId': '58993759', 'hashtagName': 'bo...",False,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,0,0,0,False,7085887049387099179,False,False,adridiaries 💌,False,False,0,MS4wLjABAAAAGyg5PhHBHSGFFuL2pbLBJiXdawbgLOtOxUidLwTLzZWOXOHXYDp5IrrFpnC...,False,digital diary\n🧸🎃🍂☕️🕯️🧦🎧\n💌 asosa4994@gmail.com,0,False,adridiaries,False,40100,308300,314,0,18900000,18900000,245,0,True,,scarlett✨,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,75,7047276707090270981,True,https://v16m.tiktokcdn-us.com/00d74933a643f72aa62b712fd83bbe04/669f3473...,hardins letter,466200,3049,1400000,8000000,19300,466162,3049,1385711,7979916,0,19332,63.36,1155035.0,"[{'Bitrate': 1155035, 'CodecType': 'h264', '

In [131]:
# export videos_df as csv
videos_df.to_csv("search_bookrec_videos.csv", index=False)

In [136]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("search_bookrec_videos.csv")

df.head()


,tiktok_url,AIGCDescription,BAInfo,adAuthorization,adLabelVersion,aigcLabelType,challenges,collected,contents,createTime,desc,digged,diversificationId,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,itemMute,officalItem,originalItem,playlistId,privateItem,secret,shareEnabled,showNotPass,stickersOnItem,stitchDisplay,stitchEnabled,textExtra,vl1,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_ttSeller,author_uniqueId,author_verified,authorStats_diggCount,authorStats_followerCount,authorStats_followingCount,authorStats_friendCount,authorStats_heart,authorStats_heartCount,authorStats_videoCount,duetInfo_duetFromId,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_playUrl,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_reflowCover,video_shareCover,video_size,video_subtitleInfos,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,videoSuggestWordsList_video_suggest_words_struct,anchors,effectStickers,poi_address,poi_category,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiId,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_type,poi_typeCode,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,https://tiktok.com/@adridiaries/video/7351077151376002347,NaN,NaN,False,0,0,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'these books > 💖 #bookrecommendations #bookstan #booktok #boo...,1711556065,these books > 💖 #bookrecommendations #bookstan #booktok #bookstagram #b...,False,10091.0,0,True,False,7351077151376002347,False,0,False,False,False,7.262052e+18,False,False,True,False,"[{'stickerText': ['books i think you\nshould read at least once\n♥️'], ...",0,True,"[{'awemeId': '', 'end': 37, 'hashtagId': '58993759', 'hashtagName': 'bo...",False,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,0,0,0,False,7085887049387099179,False,False,adridiaries 💌,False,False,0,MS4wLjABAAAAGyg5PhHBHSGFFuL2pbLBJiXdawbgLOtOxUidLwTLzZWOXOHXYDp5IrrFpnC...,False,digital diary\n🧸🎃🍂☕️🕯️🧦🎧\n💌 asosa4994@gmail.com,0,False,adridiaries,False,40100,308300,314,0,18900000,18900000,245,0,True,NaN,scarlett✨,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/cb190ca8056196846...,75,7047276707090270981,True,https://v16m.tiktokcdn-us.com/00d74933a643f72aa62b712fd83bbe04/669f3473...,hardins letter,466200,3049,1400000,8000000,19300,466162,3049,1385711,7979916,0,19332,63.36,1155035.0,"[{'Bitrate': 1155035, 'CodecType': 'h264',

In [137]:
df.shape

(167, 144)

In [138]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows()), desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', "bookrecs", "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


ERROR: [TikTok] 7394213729266715934: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7394213729266715934: Video not available, status code 10204; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@lia_faye/video/7394213729266715934


ERROR: [TikTok] 7272734235872480544: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7272734235872480544: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@authorlettyframe/video/7272734235872480544


ERROR: [TikTok] 7249889361905536302: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7249889361905536302: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@authorkittyking/video/7249889361905536302


ERROR: [TikTok] 7281378922170895662: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7281378922170895662: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@kidnappedlovewhispers0/video/7281378922170895662


In [140]:
all_videos = glob.glob("videos/bookrecs/*.mp4", recursive=True)
len(all_videos)

37

In [144]:
# Just grab the first one
filename = glob.glob("api/search/item/full/*.json")[-1]
filename

'api/search/item/full/f0a33449569c921104aab73e5ced9ce5.json'

In [145]:
from pprint import pprint

with open(filename) as fp:
    content = json.load(fp)

content.keys()

dict_keys(['url', 'query', 'headers', 'data'])

In [149]:
pprint(content['data']['item_list'][0])

{'author': {'avatarLarger': 'https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/f6c3d3b5f69a20e03665dede08e39901~c5_1080x1080.jpeg?lk3s=a5d48078&nonce=40285&refresh_token=c633a32331afa322abe25d1fd491d860&x-expires=1721779200&x-signature=6ckO16hOo%2FjDFmQTYK628IMmVAQ%3D&shp=a5d48078&shcp=b59d6b55',
            'avatarMedium': 'https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/f6c3d3b5f69a20e03665dede08e39901~c5_720x720.jpeg?lk3s=a5d48078&nonce=84718&refresh_token=bd92930dbd45c9a10a9a8037bef2e53a&x-expires=1721779200&x-signature=VNJ419YiIRaH9%2FvNrTdrx%2BeLSHE%3D&shp=a5d48078&shcp=b59d6b55',
            'avatarThumb': 'https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/f6c3d3b5f69a20e03665dede08e39901~c5_100x100.jpeg?lk3s=a5d48078&nonce=18585&refresh_token=ec90a988ff26d27ccf56dcf67b8da8c9&x-expires=1721779200&x-signature=WQaQHsxQfY3PVNIaOxfeVvtBEkc%3D&shp=a5d48078&shcp=b59d6b55',
            'commentSetting': 0,
            'download

In [150]:
# Just grab the first one
filenames = glob.glob("api/search/item/full/*.json")
filenames[:5]

['api/search/item/full/6ad3c169568f5dbd713337f877b23dc8.json', 'api/search/item/full/61cb918d67bec63333f1e3ec0eeef31e.json', 'api/search/item/full/30f4fdb455077752f71bd3d7ded6f143.json', 'api/search/item/full/2f9fb21af6921ddb3d975319cc5c6b0f.json', 'api/search/item/full/a01db4701a7f03cb6fec44aab5df89bc.json']

In [152]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['item_list'])

len(rows)

317

In [153]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,stickersOnItem,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,videoSuggestWordsList_video_suggest_words_struct,effectStickers,anchors,author_roomId
0,7318587051773791531,Books to add to your reading list for 2024 that will change your life! ...,1703991358,"[{'id': '31814052', 'title': 'readinglist', 'desc': 'Whether you're wor...",False,False,"[{'awemeId': '', 'start': 71, 'end': 83, 'hashtagName': 'readinglist', ...",False,False,False,0,False,False,False,False,True,True,True,"[{'stickerType': 4, 'stickerText': ['5 BOOKS THAT WILL CHANGE YOUR LIF...",False,False,7318587051773791531,1024,576,94,540p,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/okRGwDY7Aob...,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/d3a244bedc8...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"[, https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/4...",https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/okRGwDY7AobQNED...,371372,normal,mp4,normal,,6809298764874138629,ro.layy,Rolay,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,Fitness. Fashion. Fun.\n🌹🌹🌹🌹\n\nJOIN BOOK CLUB/READING LIST\n⬇️⬇️⬇️,False,MS4wLjABAAAAi5TPwLMvMPZY1vhBOS4bxvIBscNruEehZEgUjPlEst-jDNjH70Rn54b1Qxw...,False,False,0,False,0,0,0,False,0,7318587173228399403,original sound,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-v-...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,Rolay,True,94,,338600,25000,2139,3800000,263100,0,224,19500,745200,212,4885,745200,"[{'words': [{'word': 'the mountain is you book', 'word_id': '4878537620...",NaN,NaN,NaN
1,7319647306595257642,"If the TW list isn’t the size of a grocery list, don’t even bother 😤🫠 T...",1704238210,"[{'id': '56207580', 'title': 'bookreccomendations', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 135, 'end': 155, 'hashtagName': 'bookreccomen...",False,False,False,0,False,False,False,False,True,True,True,NaN,False,False,7319647306595257642,1024,576,15,540p,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"[, ht

In [154]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

317 total videos


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,stickersOnItem,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,videoSuggestWordsList_video_suggest_words_struct,effectStickers,anchors,author_roomId
0,https://tiktok.com/@ro.layy/video/7318587051773791531,7318587051773791531,Books to add to your reading list for 2024 that will change your life! ...,1703991358,"[{'id': '31814052', 'title': 'readinglist', 'desc': 'Whether you're wor...",False,False,"[{'awemeId': '', 'start': 71, 'end': 83, 'hashtagName': 'readinglist', ...",False,False,False,0,False,False,False,False,True,True,True,"[{'stickerType': 4, 'stickerText': ['5 BOOKS THAT WILL CHANGE YOUR LIF...",False,False,7318587051773791531,1024,576,94,540p,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/okRGwDY7Aob...,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/d3a244bedc8...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"[, https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-tx/4...",https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/okRGwDY7AobQNED...,371372,normal,mp4,normal,,6809298764874138629,ro.layy,Rolay,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,Fitness. Fashion. Fun.\n🌹🌹🌹🌹\n\nJOIN BOOK CLUB/READING LIST\n⬇️⬇️⬇️,False,MS4wLjABAAAAi5TPwLMvMPZY1vhBOS4bxvIBscNruEehZEgUjPlEst-jDNjH70Rn54b1Qxw...,False,False,0,False,0,0,0,False,0,7318587173228399403,original sound,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-v-...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,Rolay,True,94,,338600,25000,2139,3800000,263100,0,224,19500,745200,212,4885,745200,"[{'words': [{'word': 'the mountain is you book', 'word_id': '4878537620...",NaN,NaN,NaN
1,https://tiktok.com/@sinisterbookrecs/video/7319647306595257642,7319647306595257642,"If the TW list isn’t the size of a grocery list, don’t even bother 😤🫠 T...",1704238210,"[{'id': '56207580', 'title': 'bookreccomendations', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 135, 'end': 155, 'hashtagName': 'bookreccomen...",False,False,False,0,False,False,False,False,True,True,True,NaN,False,False,7319647306595257642,1024,576,15,540p,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.

In [164]:
#in column desc, check if "book" is in it
videos_df['desc'].str.contains('book', case=False).sum()

164

In [166]:
#filter only to videos with "book" in the description
videos_df = videos_df[videos_df['desc'].str.contains('book', case=False)]

In [167]:
# export videos_df as csv
videos_df.to_csv("search_bookrec2_videos.csv", index=False)

In [168]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("search_bookrec2_videos.csv")

df.head()


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,stickersOnItem,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,videoSuggestWordsList_video_suggest_words_struct,effectStickers,anchors,author_roomId
0,https://tiktok.com/@ro.layy/video/7318587051773791531,7318587051773791531,Books to add to your reading list for 2024 that will change your life! ...,1703991358,"[{'id': '31814052', 'title': 'readinglist', 'desc': ""Whether you're wor...",False,False,"[{'awemeId': '', 'start': 71, 'end': 83, 'hashtagName': 'readinglist', ...",False,False,False,0,False,False,False,False,True,True,True,"[{'stickerType': 4, 'stickerText': ['5 BOOKS THAT WILL \nCHANGE YOUR LI...",False,False,7318587051773791531,1024,576,94,540p,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/okRGwDY7Aob...,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/d3a244bedc8...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,"['', 'https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-p-0068-t...",https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/okRGwDY7AobQNED...,371372,normal,mp4,normal,NaN,6809298764874138629,ro.layy,Rolay,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,Fitness. Fashion. Fun.\n🌹🌹🌹🌹\n\nJOIN BOOK CLUB/READING LIST\n⬇️⬇️⬇️,False,MS4wLjABAAAAi5TPwLMvMPZY1vhBOS4bxvIBscNruEehZEgUjPlEst-jDNjH70Rn54b1Qxw...,False,False,0,False,0,0,0,False,0,7318587173228399403,original sound,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-v-...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/64...,Rolay,True,94,NaN,338600,25000,2139,3800000,263100,0,224,19500,745200,212,4885,745200,"[{'words': [{'word': 'the mountain is you book', 'word_id': '4878537620...",NaN,NaN,NaN
1,https://tiktok.com/@sinisterbookrecs/video/7319647306595257642,7319647306595257642,"If the TW list isn’t the size of a grocery list, don’t even bother 😤🫠 T...",1704238210,"[{'id': '56207580', 'title': 'bookreccomendations', 'desc': '', 'profil...",False,False,"[{'awemeId': '', 'start': 135, 'end': 155, 'hashtagName': 'bookreccomen...",False,False,False,0,False,False,False,False,True,True,True,NaN,False,False,7319647306595257642,1024,576,15,540p,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-p

In [169]:
df.shape

(164, 83)

In [172]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows())[100:], desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', "bookrecs2", "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


ERROR: [TikTok] 7219448114913250565: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7219448114913250565: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@fatislibrary_/video/7219448114913250565


In [174]:
all_videos = glob.glob("videos/bookrecs2/*.mp4", recursive=True)
len(all_videos)

75

# Scraping #storytime

In [178]:
# Just grab the first one
filename = glob.glob("api/search/item/full/*.json")[-1]
filename

'api/search/item/full/0e39a8dcac5151c7b8142f2814040198.json'

In [179]:
from pprint import pprint

with open(filename) as fp:
    content = json.load(fp)

content.keys()

dict_keys(['url', 'query', 'headers', 'data'])

In [180]:
pprint(content['data']['item_list'][0])

{'anchors': [{'description': 'Effects',
              'extraInfo': {'subtype': ''},
              'icon': {'urlList': ['https://p16-sg.tiktokcdn.com/obj/tiktok-obj/capcut_logo_64px_bk.png?nonce=1011&refresh_token=b5a5a9925b97178dcf8c2882909495f6&shp=b59d6b55&shcp=-']},
              'id': '5308857',
              'keyword': 'anime lashes',
              'schema': 'https://www.capcut.com/editor?scenario=tiktok&utm_source=tiktok_anchor_tool&utm_medium=tiktok_anchor&utm_campaign=71351105',
              'thumbnail': {'height': 64,
                            'urlList': ['https://p16-sg.tiktokcdn.com/obj/tiktok-obj/64px_anchor_effect3x.png?nonce=36268&refresh_token=a38b2248c334bb3ffbd8501dc935056f&shp=b59d6b55&shcp=-'],
                            'width': 64},
              'type': 28}],
 'author': {'avatarLarger': 'https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/7312485717458812971~c5_1080x1080.jpeg?lk3s=a5d48078&nonce=64898&refresh_token=7095430a8cf61305d9eb67e64c98

In [181]:
# Just grab the first one
filenames = glob.glob("api/search/item/full/*.json")
filenames[:5]

['api/search/item/full/0347db1a00b929e9d31cd54f8e4e57e3.json', 'api/search/item/full/50f8d39b62d45764287e2459726392fa.json', 'api/search/item/full/fb3bcec9684e23600b3b41d4a900f800.json', 'api/search/item/full/bef4d3960aa378ff96921c0f44d555d1.json', 'api/search/item/full/ba7cd22114136f29973c3a5e79f32723.json']

In [182]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['item_list'])

len(rows)

164

In [183]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,videoSuggestWordsList_video_suggest_words_struct,stickersOnItem,effectStickers,anchors,maskType
0,7238137459584945435,#storytime #foryou #fyp,1685260223,"[{'id': '35364', 'title': 'storytime', 'desc': 'Share a story that's go...",False,False,"[{'awemeId': '', 'start': 0, 'end': 10, 'hashtagName': 'storytime', 'ha...",False,False,False,0,False,False,False,False,True,True,True,False,False,7238137459584945435,1024,576,86,540p,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/oE...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/4d...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/b2...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,"[, https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/4db...",https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/oE3IOb...,1594630,normal,mp4,normal,,7011898750043440134,pategat,jiangyou,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,https://p77-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,https://p77-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,Hallo!\nMy friends🤓,False,MS4wLjABAAAAr6rnqVGInbr3tv087Ls32E5dCBMKHAmFq0glI42-5yXLa-ZNTADyRjIwhoD...,False,False,0,False,0,0,0,False,0,7096858059892328450,Love You So,https://sf16-sg-default.akamaized.net/obj/tos-alisg-ve-2774/df2faeb5b02...,https://p16-sg-default.akamaized.net/aweme/100x100/tos-alisg-v-2774/6aa...,https://p16-sg-default.akamaized.net/aweme/200x200/tos-alisg-v-2774/6aa...,https://p16-sg-default.akamaized.net/aweme/720x720/tos-alisg-v-2774/6aa...,The King Khan & BBQ Show,False,60,Love You So,237700,1288,5015,8400000,40900,0,33,223400,5700000,462,1497,5700000,"[{'words': [{'word': 'mom only cares about views dhar mann 3', 'word_id...",NaN,NaN,NaN,NaN
1,7208346325904018693,parte 1 | #historiaconmaquillaje #storytime #historiadeterror #ASMR #sk...,1678323921,"[{'id': '1686808082223110', 'title': 'historiaconmaquillaje', 'desc': '...",False,False,"[{'awemeId': '', 'start': 10, 'end': 32, 'hashtagName': 'historiaconmaq...",False,False,False,0,False,False,False,False,False,False,True,False,False,7208346325904018693,1030,576,176,540p,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/70c70e4b8120417...,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/c7e41524d5404ce...,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/697f2caf6286480...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,"[, https://p16-sign-va.tiktokcdn.com/tos-maliva-p-0068/c7e41524d5404ce9...",https://p16-sign-va.tiktokcdn.com/tos-maliva-p-0068/70c70e4b8120417ea20...,8

In [184]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

164 total videos


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,videoSuggestWordsList_video_suggest_words_struct,stickersOnItem,effectStickers,anchors,maskType
0,https://tiktok.com/@pategat/video/7238137459584945435,7238137459584945435,#storytime #foryou #fyp,1685260223,"[{'id': '35364', 'title': 'storytime', 'desc': 'Share a story that's go...",False,False,"[{'awemeId': '', 'start': 0, 'end': 10, 'hashtagName': 'storytime', 'ha...",False,False,False,0,False,False,False,False,True,True,True,False,False,7238137459584945435,1024,576,86,540p,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/oE...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/4d...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/b2...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,"[, https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/4db...",https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/oE3IOb...,1594630,normal,mp4,normal,,7011898750043440134,pategat,jiangyou,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,https://p77-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,https://p77-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,Hallo!\nMy friends🤓,False,MS4wLjABAAAAr6rnqVGInbr3tv087Ls32E5dCBMKHAmFq0glI42-5yXLa-ZNTADyRjIwhoD...,False,False,0,False,0,0,0,False,0,7096858059892328450,Love You So,https://sf16-sg-default.akamaized.net/obj/tos-alisg-ve-2774/df2faeb5b02...,https://p16-sg-default.akamaized.net/aweme/100x100/tos-alisg-v-2774/6aa...,https://p16-sg-default.akamaized.net/aweme/200x200/tos-alisg-v-2774/6aa...,https://p16-sg-default.akamaized.net/aweme/720x720/tos-alisg-v-2774/6aa...,The King Khan & BBQ Show,False,60,Love You So,237700,1288,5015,8400000,40900,0,33,223400,5700000,462,1497,5700000,"[{'words': [{'word': 'mom only cares about views dhar mann 3', 'word_id...",NaN,NaN,NaN,NaN
1,https://tiktok.com/@.unpocodetodo.._/video/7208346325904018693,7208346325904018693,parte 1 | #historiaconmaquillaje #storytime #historiadeterror #ASMR #sk...,1678323921,"[{'id': '1686808082223110', 'title': 'historiaconmaquillaje', 'desc': '...",False,False,"[{'awemeId': '', 'start': 10, 'end': 32, 'hashtagName': 'historiaconmaq...",False,False,False,0,False,False,False,False,False,False,True,False,False,7208346325904018693,1030,576,176,540p,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/70c70e4b8120417...,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/c7e41524d5404ce...,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/697f2caf6286480...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,"[, https://p16-sign-va.t

In [185]:
# export videos_df as csv
videos_df.to_csv("search_storytime_videos.csv", index=False)

In [186]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("search_storytime_videos.csv")

df.head()


,tiktok_url,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,itemCommentStatus,showNotPass,vl1,itemMute,privateItem,duetEnabled,stitchEnabled,shareEnabled,isAd,collected,video_id,video_height,video_width,video_duration,video_ratio,video_cover,video_originCover,video_dynamicCover,video_playAddr,video_downloadAddr,video_shareCover,video_reflowCover,video_bitrate,video_encodedType,video_format,video_videoQuality,video_encodeUserTag,author_id,author_uniqueId,author_nickname,author_avatarThumb,author_avatarMedium,author_avatarLarger,author_signature,author_verified,author_secUid,author_secret,author_ftc,author_relation,author_openFavorite,author_commentSetting,author_duetSetting,author_stitchSetting,author_privateAccount,author_downloadSetting,music_id,music_title,music_playUrl,music_coverThumb,music_coverMedium,music_coverLarge,music_authorName,music_original,music_duration,music_album,stats_diggCount,stats_shareCount,stats_commentCount,stats_playCount,stats_collectCount,duetInfo_duetFromId,authorStats_followingCount,authorStats_followerCount,authorStats_heartCount,authorStats_videoCount,authorStats_diggCount,authorStats_heart,videoSuggestWordsList_video_suggest_words_struct,stickersOnItem,effectStickers,anchors,maskType
0,https://tiktok.com/@pategat/video/7238137459584945435,7238137459584945435,#storytime #foryou #fyp,1685260223,"[{'id': '35364', 'title': 'storytime', 'desc': ""Share a story that's go...",False,False,"[{'awemeId': '', 'start': 0, 'end': 10, 'hashtagName': 'storytime', 'ha...",False,False,False,0,False,False,False,False,True,True,True,False,False,7238137459584945435,1024,576,86,540p,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/oE...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/4d...,https://p16-sign-useast2a.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/b2...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,"['', 'https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/...",https://p16-sign-useast2a.tiktokcdn.com/tos-useast2a-p-0037-aiso/oE3IOb...,1594630,normal,mp4,normal,NaN,7011898750043440134,pategat,jiangyou,https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,https://p77-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,https://p77-sign-va.tiktokcdn.com/tos-maliva-avt-0068/290152e551ea90385...,Hallo!\nMy friends🤓,False,MS4wLjABAAAAr6rnqVGInbr3tv087Ls32E5dCBMKHAmFq0glI42-5yXLa-ZNTADyRjIwhoD...,False,False,0,False,0,0,0,False,0,7096858059892328450,Love You So,https://sf16-sg-default.akamaized.net/obj/tos-alisg-ve-2774/df2faeb5b02...,https://p16-sg-default.akamaized.net/aweme/100x100/tos-alisg-v-2774/6aa...,https://p16-sg-default.akamaized.net/aweme/200x200/tos-alisg-v-2774/6aa...,https://p16-sg-default.akamaized.net/aweme/720x720/tos-alisg-v-2774/6aa...,The King Khan & BBQ Show,False,60,Love You So,237700,1288,5015,8400000,40900,0,33,223400,5700000,462,1497,5700000,"[{'words': [{'word': 'mom only cares about views dhar mann 3', 'word_id...",NaN,NaN,NaN,NaN
1,https://tiktok.com/@.unpocodetodo.._/video/7208346325904018693,7208346325904018693,parte 1 | #historiaconmaquillaje #storytime #historiadeterror #ASMR #sk...,1678323921,"[{'id': '1686808082223110', 'title': 'historiaconmaquillaje', 'desc': '...",False,False,"[{'awemeId': '', 'start': 10, 'end': 32, 'hashtagName': 'historiaconmaq...",False,False,False,0,False,False,False,False,False,False,True,False,False,7208346325904018693,1030,576,176,540p,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/70c70e4b8120417...,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/c7e41524d5404ce...,https://p16-sign-va.tiktokcdn.com/obj/tos-maliva-p-0068/697f2caf6286480...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast2a/tos-useast2a-...,"['', 'https://p16-si

In [187]:
df.shape

(164, 83)

In [190]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows()), desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', "storytime", "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


KeyboardInterrupt: 

In [3]:
all_videos = glob.glob("videos/storytime/*.mp4", recursive=True)
len(all_videos)

63